# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [24]:
!pip install chromadb langchain langchain_community langchain_openai

In [25]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [26]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY  # Ensure the environment variable is set

llm = OpenAI(temperature=0)

In [27]:
# Load the sonnets dataset
with open("sonnets.txt", "r", encoding="utf-8") as file:
    sonnets = file.read()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
documents = text_splitter.create_documents([sonnets])


In [28]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "sonnets.txt")

In [29]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

In [30]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [31]:
from langchain_community.document_loaders import WebBaseLoader

In [32]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [33]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2122, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1049, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 2321, which is longer than the specified 1000


## Create the Agent

In [34]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [35]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [36]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.invoke(
    "What are the main themes in Shakespeare's sonnets?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation:  I don't know.
Thought: I should try rephrasing the question.
Action: State of Union QA System
Action Input: "What was Biden's mention of Ketanji Brown Jackson in the state of the union address?"
Observation:  I don't know.
Thought: I should try using different keywords.
Action: State of Union QA System
Action Input: "What was Biden's statement about Ketanji Brown Jackson in the state of the union address?"
Observation:  I don't know.
Thought: I should try using the full name of the person.
Action: State of Union QA System
Action Input: "What was Biden's statement about Ketanji Brown Jackson in the state of the union address?"
Observation:  I don't know.
Thought: I should try using the full name of the person and the word "mention".
Actio

{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [ ]:
agent.invoke("Which sonnet talks about love and time?")



> Entering new AgentExecutor chain...
 You should always think about the differences between ruff and flake8 before deciding which one to use.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff and Flake8 are both linters for Python code, but they have some key differences. Ruff uses different rule codes and prefixes than Flake8, which helps minimize conflicts across plugins and allows for easier toggling of individual plugins. Ruff also has a larger rule set, with over 800 rules compared to Flake8's ~409 rules. However, Ruff does not support custom lint rules, while Flake8 does. Additionally, Ruff is not a "pure" drop-in replacement for Flake8, as they enforce different sets of rules. Ruff also has a formatter that is designed to be a drop-in replacement for Black, while Flake8 does not have a built-in formatter.
Thought: I now know the final answer.
Final Answer: Ruff has a larger rule set and better compatibility with plugin

{'input': 'Why use ruff over flake8?',
 'output': 'Ruff has a larger rule set and better compatibility with plugins, but Flake8 allows for custom lint rules and has a built-in formatter. The decision between the two ultimately depends on the specific needs and preferences of the user.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [ ]:
tools = [
    Tool(
        name="Shakespeare Sonnets QA System",
        func=sonnets_qa.run,  # Make sure this matches your RetrievalQA variable
        description="Useful for answering literary questions about Shakespeare's sonnets. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="Useful for when you need to answer questions about Ruff (a Python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]


In [40]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.invoke(
    "What are the main themes in Shakespeare's sonnets?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation:  I don't know.


> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': " I don't know."}

In [ ]:
agent.invoke("Which sonnet talks about love and time?")



> Entering new AgentExecutor chain...
 You should always think about the differences between ruff and flake8 before deciding which one to use.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff and Flake8 are both linters for Python code, but they have some key differences. Ruff uses different rule codes and prefixes than Flake8, which helps minimize conflicts across plugins and allows for easier toggling of individual plugins. Ruff also has a larger rule set, with over 800 rules compared to Flake8's ~409 rules. However, Ruff does not support custom lint rules, while Flake8 does. Additionally, Ruff is not a "pure" drop-in replacement for Flake8, as they enforce different sets of rules. Ruff also has a formatter that is designed to be a drop-in replacement for Black, while Flake8 does not have a built-in formatter.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': ' Ruff and Flake8 are both linters for Python code, but they have some key differences. Ruff uses different rule codes and prefixes than Flake8, which helps minimize conflicts across plugins and allows for easier toggling of individual plugins. Ruff also has a larger rule set, with over 800 rules compared to Flake8\'s ~409 rules. However, Ruff does not support custom lint rules, while Flake8 does. Additionally, Ruff is not a "pure" drop-in replacement for Flake8, as they enforce different sets of rules. Ruff also has a formatter that is designed to be a drop-in replacement for Black, while Flake8 does not have a built-in formatter.'}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [ ]:
tools = [
    Tool(
        name="Shakespeare Sonnets QA System",
        func=sonnets_qa.run,  # Make sure this variable is correct
        description="Useful for answering questions about Shakespeare's sonnets. Input should be a fully formed literary question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="Useful for answering questions about Ruff (a Python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]


In [44]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the sonnets mention that tool in Sonnet 18?"
)



> Entering new AgentExecutor chain...
 I should use the Ruff QA System to answer this question.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff does not support source.organizeImports and source.fixAll code actions in Jupyter Notebooks (notebook.codeActionsOnSave in VS Code).
Thought: I should use the State of Union QA System to check if the president mentioned this tool.
Action: State of Union QA System
Action Input: Did the president mention that tool in the state of the union?
Observation:  I don't know.
Thought: I should use the Ruff QA System again to see if it has any information on the president mentioning this tool.
Action: Ruff QA System
Action Input: Did the president mention that tool in the state of the union?
Observation:  I don't know.
Thought: I should use the State of Union QA System again to see if it has any information on the president mentioning this tool.
Action: State of Union QA System
Action Input:

{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?',
 'output': "I don't know."}